In [9]:
# --- Upload the raw data ---

import pandas as pd

# load the three raw data files
esg_df = pd.read_csv("sustainability-economic-performance/data/raw/esg-economic-data.csv")
gdp_df = pd.read_csv("sustainability-economic-performance/data/raw/gdp-inflation-fdi-data.csv")
class_df = pd.read_excel("sustainability-economic-performance/data/raw/country-classification.xlsx")

# print a preview of each one to see what it looks like
print("ESG + other indicators:")
display(esg_df.head())

print("\nGDP, Inflation, FDI:")
display(gdp_df.head())

print("\nCountry classification:")
display(class_df.head())

ESG + other indicators:


,Country Name,Country Code,Series Name,Series Code,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.0535570038181695,0.0536689767118985,0.0656292600512945,0.0727226251084929,0.0548502176552485,0.0803829463559013,...,0.283032202188285,0.297282459170618,0.267860693638487,0.28083718384984,0.298624727257862,0.296882768311101,0.222925457048673,..,..,..
1,Afghanistan,AFG,Fossil fuel energy consumption (% of total),EG.USE.COMM.FO.ZS,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,Afghanistan,AFG,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,45,45.6,37.8,36.7,44.2,33.9,...,19.1,17.7,20.2,19.5,18.3,18.9,18.2,20,20,..
3,Afghanistan,AFG,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,93.5373177008865,93.5373177008865,93.5374497028126,80.6308604401597,62.8649359905699,63.0139791405013,...,85.4614747024191,87.5782394741968,87.7612654613868,86.8645836678334,82.6719956888809,87.8403578186823,87.123785906179,78.2344746063241,..,..
4,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,EN.ATM.NOXE.PC,0.172754004045737,0.1515261920459,0.166326426410708,0.162478786281594,0.155604365523979,0.156654089498904,...,0.155190837374727,0.149477465644416,0.145343704975578,0.147495689770513,0.127109966516379,0.122765320540897,0.124482055213165,..,..,..



GDP, Inflation, FDI:


,Country Name,Country Code,Series Name,Series Code,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,308.318269746638,277.118051443941,338.139973643387,346.071627096223,338.637273888197,363.640141436773,...,575.146245808546,565.569730408751,563.872336723147,562.769574140988,553.125151688293,557.861533207459,527.834554499306,408.625855217403,377.665627080705,378.06630312259
1,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,..,..,..,..,..,..,...,4.67399603536345,-0.661709164713771,4.38389195513915,4.97595150553836,0.626149149168802,2.30237251516839,5.60188791482222,5.1332034082496,13.7121023720065,-4.64470870797776
2,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",BX.KLT.DINV.WD.GD.ZS,0.00482760061734038,0.0241685679088889,1.30694987042884,1.27849325196424,3.577104200584,4.36867310434287,...,0.209664794672203,0.88400046336123,0.516606084523093,0.274796791572807,0.661572217787196,0.124495985291258,0.0649939571808755,0.144466946606861,..,..
3,Albania,ALB,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,1991.91435867116,2188.91854451375,2297.10853513924,2428.68934653563,2567.29763687121,2712.86983830331,...,3883.63262828191,3981.72662261867,4143.98988316994,4283.98262744437,4452.23714652476,4563.4673625169,4437.65346927974,4880.72346162808,5178.88431481499,5444.93000074979
4,Albania,ALB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,..,..,..,..,..,..,...,1.62586504402611,1.89617402592365,1.27543168367414,1.98666133171195,2.02805963071135,1.41109078954247,1.62088661717011,2.04147163139547,6.72520271530052,4.75976421930108



Country classification:


,Economy,Code,Region,Income group,Lending category
0,Afghanistan,AFG,"Middle East, North Africa, Afghanistan & Pakistan",Low income,IDA
1,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD
2,Algeria,DZA,"Middle East, North Africa, Afghanistan & Pakistan",Upper middle income,IBRD
3,American Samoa,ASM,East Asia & Pacific,High income,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN


In [10]:
# --- Clean ESG dataset structure ---

# drop series code column (not useful for the project)
esg_clean = esg_df.drop(columns=["Series Code"])

# reshape the dataset from wide format to long format
year_cols = [col for col in esg_clean.columns if "[" in col] 

esg_long = esg_clean.melt(
    id_vars=["Country Name", "Country Code", "Series Name"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)

# rename the "Series Name" column to "Indicator"
esg_long = esg_long.rename(columns={"Series Name": "Indicator"})

# clean Year column name: keep only the 4 digits
esg_long["Year"] = esg_long["Year"].str.slice(0, 4)

# convert Year to integer
esg_long["Year"] = esg_long["Year"].astype(int)

# display the first 10 rows to see if it worked
esg_long.head(10)

,Country Name,Country Code,Indicator,Year,Value
0,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.0535570038181695
1,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,..
2,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45
3,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.5373177008865
4,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754004045737
5,Afghanistan,AFG,Methane emissions (metric tons of CO2 equivale...,2000,0.474923530452337
6,Afghanistan,AFG,"Unemployment, total (% of total labor force) (...",2000,7.935
7,Afghanistan,AFG,Economic and Social Rights Performance Score,2000,..
8,Afghanistan,AFG,Control of Corruption: Estimate,2000,-1.27172422409058
9,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,2000,-2.43896889686584


In [11]:
# --- Clean GDP/Inflation/FDI dataset structure ---

# drop series code column (not useful for the project)
gdp_clean = gdp_df.drop(columns=["Series Code"])

# reshape the dataset from wide format to long format
year_cols_gdp = [col for col in gdp_clean.columns if "[" in col]

gdp_long = gdp_clean.melt(
    id_vars=["Country Name", "Country Code", "Series Name"],
    value_vars=year_cols_gdp,
    var_name="Year",
    value_name="Value"
)

# rename the "Series Name" column to "Indicator"
gdp_long = gdp_long.rename(columns={"Series Name": "Indicator"})

# clean Year column name: keep only the 4 digits
gdp_long["Year"] = gdp_long["Year"].str.slice(0, 4)

# convert Year to integer
gdp_long["Year"] = gdp_long["Year"].astype(int)

# display the first 10 rows to see if it worked
gdp_long.head(10)

,Country Name,Country Code,Indicator,Year,Value
0,Afghanistan,AFG,GDP per capita (constant 2015 US$),2000,308.318269746638
1,Afghanistan,AFG,"Inflation, consumer prices (annual %)",2000,..
2,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",2000,0.00482760061734038
3,Albania,ALB,GDP per capita (constant 2015 US$),2000,1991.91435867116
4,Albania,ALB,"Inflation, consumer prices (annual %)",2000,..
5,Albania,ALB,"Foreign direct investment, net inflows (% of GDP)",2000,3.9893207113692
6,Algeria,DZA,GDP per capita (constant 2015 US$),2000,3553.32420521991
7,Algeria,DZA,"Inflation, consumer prices (annual %)",2000,..
8,Algeria,DZA,"Foreign direct investment, net inflows (% of GDP)",2000,0.511220957154505
9,Andorra,AND,GDP per capita (constant 2015 US$),2000,34113.8173771495


In [12]:
# --- Add a Category column to the ESG dataset ---

def assign_category(indicator_name):
    indicator_name_lower = str(indicator_name).lower()
    
    # Environmental
    if "co2" in indicator_name_lower:
        return "Environmental"
    if "fossil" in indicator_name_lower:
        return "Environmental"
    if "renewable" in indicator_name_lower:
        return "Environmental"
    if "methane" in indicator_name_lower:
        return "Environmental"
    if "nitrous" in indicator_name_lower:
        return "Environmental"
    
    # Social
    if "unemployment" in indicator_name_lower:
        return "Social"
    if "gini" in indicator_name_lower:
        return "Social"
    if "rights" in indicator_name_lower:
        return "Social"

    # Governance
    if "corruption" in indicator_name_lower:
        return "Governance"
    if "political" in indicator_name_lower:
        return "Governance"

    # Economic
    if "gdp" in indicator_name_lower:
        return "Economic"
    if "expenditure" in indicator_name_lower:
        return "Economic"

    return "Other"

# apply to ESG data
esg_long["Category"] = esg_long["Indicator"].apply(assign_category)

# display a table showing each indicator in the dataset and its assigned category
esg_long[["Indicator", "Category"]].drop_duplicates().sort_values("Indicator")

,Indicator,Category
0,CO2 emissions (metric tons per capita),Environmental
8,Control of Corruption: Estimate,Governance
7,Economic and Social Rights Performance Score,Social
1,Fossil fuel energy consumption (% of total),Environmental
11,GDP growth (annual %),Economic
12,Gini index,Social
5,Methane emissions (metric tons of CO2 equivale...,Environmental
4,Nitrous oxide emissions (metric tons of CO2 eq...,Environmental
9,Political Stability and Absence of Violence/Te...,Governance
3,Renewable electricity output (% of total elect...,Environmental


In [13]:
# --- Add a Category column to the gdp-inflation-fdi dataset ---

def assign_economic_category(indicator_name):
    indicator_name_lower = str(indicator_name).lower()

    #Economic
    if "gdp" in indicator_name_lower:
        return "Economic"
    if "inflation" in indicator_name_lower:
        return "Economic"
    if "foreign direct investment" in indicator_name_lower:
        return "Economic"
    if "research" in indicator_name_lower or "r&d" in indicator_name_lower:
        return "Economic"

    return "Other"

# apply to gdp-inflation-fdi data
gdp_long["Category"] = gdp_long["Indicator"].apply(assign_economic_category)

# display a table showing each indicator in the dataset and its assigned category
gdp_long[["Indicator", "Category"]].drop_duplicates().sort_values("Indicator")

,Indicator,Category
2,"Foreign direct investment, net inflows (% of GDP)",Economic
0,GDP per capita (constant 2015 US$),Economic
1,"Inflation, consumer prices (annual %)",Economic
579,NaN,Other


In [14]:
# drop rows where Series Name shows "Other" (NaN)
esg_long = esg_long.dropna(subset=["Indicator"])
gdp_long = gdp_long.dropna(subset=["Indicator"])

# display the same tables again to see if it worked
from IPython.display import display

display(esg_long[["Indicator", "Category"]].drop_duplicates().sort_values("Indicator"))
display(gdp_long[["Indicator", "Category"]].drop_duplicates().sort_values("Indicator"))

,Indicator,Category
0,CO2 emissions (metric tons per capita),Environmental
8,Control of Corruption: Estimate,Governance
7,Economic and Social Rights Performance Score,Social
1,Fossil fuel energy consumption (% of total),Environmental
11,GDP growth (annual %),Economic
12,Gini index,Social
5,Methane emissions (metric tons of CO2 equivale...,Environmental
4,Nitrous oxide emissions (metric tons of CO2 eq...,Environmental
9,Political Stability and Absence of Violence/Te...,Governance
3,Renewable electricity output (% of total elect...,Environmental


,Indicator,Category
2,"Foreign direct investment, net inflows (% of GDP)",Economic
0,GDP per capita (constant 2015 US$),Economic
1,"Inflation, consumer prices (annual %)",Economic


In [15]:
# --- Merge ESG and gdp-inflation-fdi datasets ---

# first add a source label column that tells from which dataset an indicator comes from (just in case)
esg_long["Source"] = "ESG"
gdp_long["Source"] = "Economic"

# make columns order consistent
common_cols = ["Country Name", "Country Code", "Indicator", "Year", "Value", "Category", "Source"]

esg_long = esg_long[common_cols]
gdp_long = gdp_long[common_cols]

# combine the two datasets into a single long dataframe
all_long = pd.concat([esg_long, gdp_long], ignore_index=True)

# display the first 15 rows of the new merged dataset
all_long.head(15)

,Country Name,Country Code,Indicator,Year,Value,Category,Source
0,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.0535570038181695,Environmental,ESG
1,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,..,Environmental,ESG
2,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45,Environmental,ESG
3,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.5373177008865,Environmental,ESG
4,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754004045737,Environmental,ESG
5,Afghanistan,AFG,Methane emissions (metric tons of CO2 equivale...,2000,0.474923530452337,Environmental,ESG
6,Afghanistan,AFG,"Unemployment, total (% of total labor force) (...",2000,7.935,Social,ESG
7,Afghanistan,AFG,Economic and Social Rights Performance Score,2000,..,Social,ESG
8,Afghanistan,AFG,Control of Corruption: Estimate,2000,-1.27172422409058,Governance,ESG
9,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,2000,-2.43896889686584,Governance,ESG


In [16]:
# sort the data by Country Code
all_long = all_long.sort_values(["Country Code", "Year", "Category"]).reset_index(drop=True)

# display the first 20 rows
all_long.head(20)

,Country Name,Country Code,Indicator,Year,Value,Category,Source
0,Afghanistan,AFG,Research and development expenditure (% of GDP),2000,..,Economic,ESG
1,Afghanistan,AFG,GDP growth (annual %),2000,..,Economic,ESG
2,Afghanistan,AFG,GDP per capita (constant 2015 US$),2000,308.318269746638,Economic,Economic
3,Afghanistan,AFG,"Inflation, consumer prices (annual %)",2000,..,Economic,Economic
4,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",2000,0.00482760061734038,Economic,Economic
5,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.0535570038181695,Environmental,ESG
6,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,..,Environmental,ESG
7,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45,Environmental,ESG
8,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.5373177008865,Environmental,ESG
9,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754004045737,Environmental,ESG


In [17]:
# --- Prepare country-classification table ---

# drop "lending category" column and rename remaining columns to match the other (merged) dataset
class_clean = class_df.rename(
    columns={
        "Economy": "Country Name",
        "Code": "Country Code",
        "Income group": "Income Group",
        "Lending category": "Lending Category"
    }
)[["Country Code", "Country Name", "Region", "Income Group"]]

# display the first 15 rows
class_clean.head(15)

,Country Code,Country Name,Region,Income Group
0,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
1,ALB,Albania,Europe & Central Asia,Upper middle income
2,DZA,Algeria,"Middle East, North Africa, Afghanistan & Pakistan",Upper middle income
3,ASM,American Samoa,East Asia & Pacific,High income
4,AND,Andorra,Europe & Central Asia,High income
5,AGO,Angola,Sub-Saharan Africa,Lower middle income
6,ATG,Antigua and Barbuda,Latin America & Caribbean,High income
7,ARG,Argentina,Latin America & Caribbean,Upper middle income
8,ARM,Armenia,Europe & Central Asia,Upper middle income
9,ABW,Aruba,Latin America & Caribbean,High income


In [18]:
# --- Merge all datasets into one unique dataset

panel_long = all_long.merge(
    class_clean,
    on="Country Code",
    how="left"
)

# display the first 20 rows
panel_long.head(10)

,Country Name_x,Country Code,Indicator,Year,Value,Category,Source,Country Name_y,Region,Income Group
0,Afghanistan,AFG,Research and development expenditure (% of GDP),2000,..,Economic,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
1,Afghanistan,AFG,GDP growth (annual %),2000,..,Economic,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
2,Afghanistan,AFG,GDP per capita (constant 2015 US$),2000,308.318269746638,Economic,Economic,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
3,Afghanistan,AFG,"Inflation, consumer prices (annual %)",2000,..,Economic,Economic,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
4,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",2000,0.00482760061734038,Economic,Economic,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
5,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.0535570038181695,Environmental,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
6,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,..,Environmental,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
7,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45,Environmental,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
8,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.5373177008865,Environmental,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income
9,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754004045737,Environmental,ESG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income


In [19]:
# clean up duplicate country name columns
if "Country Name_x" in panel_long.columns:
    panel_long = panel_long.rename(columns={"Country Name_x": "Country Name"})
    panel_long = panel_long.drop(columns=["Country Name_y"])

# display the first 20 rows
panel_long.head(10)

,Country Name,Country Code,Indicator,Year,Value,Category,Source,Region,Income Group
0,Afghanistan,AFG,Research and development expenditure (% of GDP),2000,..,Economic,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
1,Afghanistan,AFG,GDP growth (annual %),2000,..,Economic,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
2,Afghanistan,AFG,GDP per capita (constant 2015 US$),2000,308.318269746638,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
3,Afghanistan,AFG,"Inflation, consumer prices (annual %)",2000,..,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
4,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",2000,0.00482760061734038,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
5,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.0535570038181695,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
6,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,..,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
7,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
8,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.5373177008865,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
9,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754004045737,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income


In [20]:
# Convert Value column to numeric (turn ".." into NaN)
panel_long["Value"] = pd.to_numeric(panel_long["Value"], errors="coerce")

# display the first 20 rows
panel_long.head(10)

,Country Name,Country Code,Indicator,Year,Value,Category,Source,Region,Income Group
0,Afghanistan,AFG,Research and development expenditure (% of GDP),2000,NaN,Economic,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
1,Afghanistan,AFG,GDP growth (annual %),2000,NaN,Economic,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
2,Afghanistan,AFG,GDP per capita (constant 2015 US$),2000,308.318270,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
3,Afghanistan,AFG,"Inflation, consumer prices (annual %)",2000,NaN,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
4,Afghanistan,AFG,"Foreign direct investment, net inflows (% of GDP)",2000,0.004828,Economic,Economic,"Middle East, North Africa, Afghanistan & Pakistan",Low income
5,Afghanistan,AFG,CO2 emissions (metric tons per capita),2000,0.053557,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
6,Afghanistan,AFG,Fossil fuel energy consumption (% of total),2000,NaN,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
7,Afghanistan,AFG,Renewable energy consumption (% of total final...,2000,45.000000,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
8,Afghanistan,AFG,Renewable electricity output (% of total elect...,2000,93.537318,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income
9,Afghanistan,AFG,Nitrous oxide emissions (metric tons of CO2 eq...,2000,0.172754,Environmental,ESG,"Middle East, North Africa, Afghanistan & Pakistan",Low income


In [21]:
# save the (full) merged dataset
output_path = "sustainability-economic-performance/data/processed/panel_full_unfiltered.csv"
panel_long.to_csv(output_path, index=False)